Starting from a population of cells at different stages of development process, trajectory inference algorithms aim to reconstruct the developmental sequence of transcriptional changes leading to potential cell fates.

Unspliced pre-mRNAs and mature spliced mRNAs, the former can detectable by presence of introns

In [2]:
import pickle
import tensorflow_probability as tfp
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import *
import anndata
from scipy.sparse import csr_matrix, issparse, coo_matrix
import seaborn as sns
from scanpy import Neighbors


In [3]:
adata = anndata.read("../processed_pancreas.hdf5")

In [4]:
def get_weight(x, y=None, perc=95):
    """
    gets the masking weights for gene count in upper and lower percentile 
    does it by normalizing the each gene vector by the max of each gene vector uses clipping 
    and then creates a mask selecting only the top percentile for computing the constant valued 
    degradation rate 
    """
    xy_norm = np.array(x.A if issparse(x) else x)
    if y is not None:
        if issparse(y):
            y = y.A
        xy_norm = xy_norm / np.clip(np.max(xy_norm, axis=0), 1e-3, None)
        xy_norm += y / np.clip(np.max(y, axis=0), 1e-3, None)
    if isinstance(perc, int):
        weights = xy_norm >= np.percentile(xy_norm, perc, axis=0)
    else:
        lb, ub = np.percentile(xy_norm, perc, axis=0)
        weights = (xy_norm <= lb) | (xy_norm >= ub)
    return weights


shape is cell x gene

In [5]:
adata

AnnData object with n_obs × n_vars = 3696 × 1945
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition'
    var: 'highly_variable_genes', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca', 'velocity_graph', 'velocity_graph_neg', 'velocity_params'
    obsm: 'X_pca', 'X_umap', 'velocity_umap'
    layers: 'Ms', 'Mu', 'spliced', 'unspliced', 'velocity'
    obsp: 'connectivities', 'distances'

In [6]:
spliced = adata.layers["Ms"]
unspliced = adata.layers["Mu"]

In [7]:
w =  csr_matrix(get_weight(spliced, unspliced))

In [8]:
x, y = w.multiply(spliced).tocsr(), w.multiply(unspliced).tocsr()

In [9]:
x = x.A
y = y.A

## Kinetics are - 
$\frac{du(t)}{dt} = \alpha^k(t) - \beta u(t)$  
$\frac{ds(t)}{dt} = \beta u(t) - \gamma s(t)$


Now at s.s. the term $\frac{ds(t)}{dt} = 0$ this implies that $\beta u(t) - \gamma s(t) = 0$. $\gamma$ represents the degradation rate of mRNA and hence we can infer $\gamma$ from this condition. For this we take the gene values in the upper percentile as they are the ones most likely to be at a steady state and use linear regression to infer the $\gamma$ values from those list of values  

## Note on linear regression 

$Y = \beta*X + \epsilon$  In our case $Y \sim u $ and $X \sim s$ and we actually infer $\frac{\gamma}{\beta}$

A closed form solution is 

$\beta = (X^T X)^{-1} X^T Y$


To derive the formula used just write the $err = \sum_{i}^{nobs} (y_i - w_i x_i)^2$ and take derivatives w.r.t $w_i$

We can include a positive offset for the basal transcription rate in each cell 

In [10]:
xx = np.multiply(x, x).sum(axis = 0)
xy = np.multiply(x, y).sum(axis = 0) 

In [11]:
gamma = xy/xx


### compare with scvelo implementation 


In [12]:
gamma_scanpy = np.asarray(adata.var["velocity_gamma"].tolist())
np.allclose(gamma, gamma_scanpy)

True

### velocity computations 

the velocity is given by the deviation of the ratio of unspliced and spliced from the ss ratio

$\nu_i = u_i - \gamma_i' s_i$

In [12]:
velocity = unspliced - gamma*spliced ## this is ds/dt
alpha = unspliced ## this is du/dt
r2 = R_squared(velocity, total=unspliced - unspliced.mean(0))

### compare with scvelo implementation 

In [15]:
np.allclose(velocity, adata.layers["velocity"])

True

In [17]:
### filter genes by min likelihood comparison

In [18]:
min_r2 = 1e-2
min_ratio = 1e-2


In [19]:
def mask_genes(r2, gamma, min_r2, min_ratio, spliced, unspliced):
    velocity_mask = (
                (r2 > min_r2)
                & (gamma > min_ratio)
                & (np.max(spliced > 0, 0) > 0)
                & (np.max(unspliced > 0, 0) > 0)
            )
    return velocity_mask

In [20]:
mask = mask_genes(r2, gamma, min_r2, min_ratio, spliced, unspliced)

In [387]:
## filtering step using min likelihood threshold 

In [24]:
vgenes = adata.var["velocity_genes"].values

In [26]:
len(vgenes) == sum(vgenes==mask)

True

In [ ]:

kwargs = dict(frameon=False, ylabel='cell cycle genes')
scv.pl.scatter(adata, "", **kwargs)

NameError: name 'adata' is not defined